### s000 environment setup

In [0]:
# libraries
import os

import pyspark
from pyspark import sql
from pyspark.sql import functions as f
from pyspark.sql import Window

In [0]:
%run "./utils_spark_df"

In [0]:
%run "./utils_stratified_sampling"

In [0]:
%run "./sc_utils_stratified_sampling"

In [0]:
df_campaign_cand_control = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/250123_Samsung_NPI_Consumer_email_regen_control')
df_campaign_cand_target = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/250123_Samsung_NPI_Consumer_email_regen_target')

In [0]:
df_prm_ifp_bill = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/prm_ifp_main_on_bill')
df_prm_ifp_srvc = spark.read.format('delta').load('dbfs:/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/prm_ifp_main_on_service')

In [0]:
df_ifp_bill = (
    df_prm_ifp_bill
    .filter(f.col('ifp_type') == 'device')
    .filter(f.col('ifp_term_start_date').isNotNull())
    .select('fs_cust_id', 'fs_acct_id', 'fs_ifp_id', 'ifp_model', 'ifp_term_start_date', 'ifp_term_end_date', 'ifp_sales_channel', 'ifp_sales_channel_branch', 'ifp_sales_channel_group')
    .distinct()
)

df_ifp_srvc=(
    df_prm_ifp_srvc
    .filter(f.col('ifp_type') == 'device')
    .filter(f.col('ifp_term_start_date').isNotNull())
    .select('fs_cust_id', 'fs_acct_id', 'fs_ifp_id', 'ifp_model','ifp_term_start_date', 'ifp_term_end_date', 'ifp_sales_channel', 'ifp_sales_channel_branch', 'ifp_sales_channel_group')
    .distinct()
)

df_ifp_all=(
    df_ifp_bill
    .union(df_ifp_srvc)
)

In [0]:
vt_send_date = '2025-01-23'

In [0]:
display(df_campaign_cand_target.limit(10))

In [0]:
display(df_ifp_all.limit(10))

In [0]:
display(
    df_campaign_cand_control
    .join(df_ifp_all, ['fs_cust_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= f.lit(vt_send_date))
    .filter(f.col('ifp_term_start_date') <= f.add_months(f.lit(vt_send_date), 1))
    .groupBy('ifp_model')
    .agg(
        f.countDistinct('fs_cust_id')
        , f.min('ifp_term_start_date')
        , f.max('ifp_term_start_date')
    )
)


In [0]:
display(
    df_campaign_cand_target
    .join(df_ifp_all, ['fs_cust_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= f.lit(vt_send_date))
    .filter(f.col('ifp_term_start_date') <= f.add_months(f.lit(vt_send_date), 1))
    .groupBy('ifp_model')
    .agg(
        f.countDistinct('fs_cust_id')
        , f.min('ifp_term_start_date')
        , f.max('ifp_term_start_date')
    )
)


In [0]:
display(
    df_campaign_cand_control
    .join(df_ifp_all, ['fs_cust_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= f.lit(vt_send_date))
    .filter(f.col('ifp_term_start_date') <= f.add_months(f.lit(vt_send_date), 1))
    .agg(
        f.countDistinct('fs_cust_id')
        , f.min('ifp_term_start_date')
        , f.max('ifp_term_start_date')
    )
)


In [0]:
display(df_campaign_cand_target
        .agg(f.countDistinct('fs_cust_id'))
        )

In [0]:
display(df_campaign_cand_control
        .agg(f.countDistinct('fs_cust_id'))
        )

In [0]:
print("control")
display(
    df_campaign_cand_control
    .agg(
        f.countDistinct("fs_cust_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)


print("target")
display(
    df_campaign_cand_target
    .agg(
        f.countDistinct("fs_cust_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)